**How to read `.nur` files**
============================
In this notebook, we will see how to access the information in a `.nur` file generated by `NuRadioMC`.
If you haven't already, you can generate the sample file used in this notebook using 
[the simulation example](W01-simulate-neutrino-detector.ipynb).

To read in events, we will use the [`eventReader`](https://nu-radio.github.io/NuRadioMC/NuRadioReco/apidoc/NuRadioReco.modules.io.eventReader.html#NuRadioReco.modules.io.eventReader.eventReader) class.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from NuRadioReco.utilities import units # we use the unit system provided by NuRadioReco
from NuRadioReco.modules.io import eventReader

reader = eventReader.eventReader()

The [`eventReader`](https://nu-radio.github.io/NuRadioMC/NuRadioReco/apidoc/NuRadioReco.modules.io.eventReader.html#NuRadioReco.modules.io.eventReader.eventReader) has a `begin()` which expects a (list of) filename(s) to read in.
You can then access some top-level parameters quickly by using `reader.get_header()`, or open the events one by one by using the `run()` method.

In [ ]:
filename = '1e19_n1e3_output.nur' # one can also supply a list of filenames
reader.begin(filename)

# the header allows quick array-form access
# to some top-level parameters (amplitudes, trigger times...)
header = reader.get_header()

In [ ]:
### reader.run() is an iterator over all events:
# for event in reader.run():
#     (do some analysis for each `event`)
#
### For now, we will just look at the first event in the file instead

event = next(reader.run())

print(f"Event with run number {event.get_run_number()} and event_id {event.get_id()}")


**Accessing the different objects**
-------------------------------------------------

![Event structure](https://nu-radio.github.io/NuRadioMC/_images/event_structure.png)

An [`Event`](https://nu-radio.github.io/NuRadioMC/NuRadioReco/apidoc/NuRadioReco.framework.event.html#NuRadioReco.framework.event.Event) contains multiple sub-objects, which can be obtained using `.get_<>` methods.
- The [`Event`](https://nu-radio.github.io/NuRadioMC/NuRadioReco/apidoc/NuRadioReco.framework.event.html#NuRadioReco.framework.event.Event) itself contains mostly id information, i.e. run number and event id.
- The [`Particle`](https://nu-radio.github.io/NuRadioMC/NuRadioReco/apidoc/NuRadioReco.framework.particle.html#module-NuRadioReco.framework.particle) class contains the description of the parent particle(s) that gave rise to the event, e.g. particle energy, direction, flavour, interaction type...
- The [`RadioShower`](https://nu-radio.github.io/NuRadioMC/NuRadioReco/apidoc/NuRadioReco.framework.radio_shower.html#NuRadioReco.framework.radio_shower.RadioShower) class has some overlap with the [`Particle`](https://nu-radio.github.io/NuRadioMC/NuRadioReco/apidoc/NuRadioReco.framework.particle.html#module-NuRadioReco.framework.particle) class, but contains information on the shower (rather than particle) energy, shower core position, ...
- The [`Station`](https://nu-radio.github.io/NuRadioMC/NuRadioReco/apidoc/NuRadioReco.framework.station.html#module-NuRadioReco.framework.station) contains 'station'-level quantities, for example triggers, maximum amplitude over all channels, etc. 
  It also stores most reconstructed quantities - in our example case, these will be undefined (still). It is also used to access:
    - channel-level data through the `get_channel` method. The [`Channel`](https://nu-radio.github.io/NuRadioMC/NuRadioReco/apidoc/NuRadioReco.framework.channel.html#module-NuRadioReco.framework.channel) is where the voltage waveforms (as recorded by the simulated detector) live.
    - the (3D) [`ElectricField`](https://nu-radio.github.io/NuRadioMC/NuRadioReco/apidoc/NuRadioReco.framework.electric_field.html#module-NuRadioReco.framework.electric_field)s. Note that a [`Station`](https://nu-radio.github.io/NuRadioMC/NuRadioReco/apidoc/NuRadioReco.framework.station.html#module-NuRadioReco.framework.station) only includes **measured** or **reconstructed** data; it may initially not contain any electric fields, as these first have to be reconstructed from the measured voltage traces stored in the [`Channel`](https://nu-radio.github.io/NuRadioMC/NuRadioReco/apidoc/NuRadioReco.framework.channel.html#module-NuRadioReco.framework.channel)s.
    - the [`Trigger`](https://nu-radio.github.io/NuRadioMC/NuRadioReco/apidoc/NuRadioReco.framework.trigger.html#NuRadioReco.framework.trigger.Trigger)(s), which contain information about the trigger that was used, including e.g. trigger type, thresholds, and trigger times.
    - the [`SimStation`](https://nu-radio.github.io/NuRadioMC/NuRadioReco/apidoc/NuRadioReco.framework.sim_station.html#module-NuRadioReco.framework.sim_station) (roughly speaking, a copy of the station but with MC-level truth information). This stores noiseless copies of the signals generated by the simulated neutrino/cosmic ray/emitter, as well as the simulated electric fields at each antenna.


Let's first look at the [`Station`](https://nu-radio.github.io/NuRadioMC/NuRadioReco/apidoc/NuRadioReco.framework.station.html#module-NuRadioReco.framework.station), which contains the recorded voltage traces. We will also use the [`Trigger`](https://nu-radio.github.io/NuRadioMC/NuRadioReco/apidoc/NuRadioReco.framework.trigger.html#NuRadioReco.framework.trigger.Trigger) to look at when the trigger fired.

In [ ]:
# for an Event with multiple stations, one has to explicitly specify the station_id
# Here, we only have one station, so this is optional
station = event.get_station()

# get the first trigger which fired.
# We can also get a dictionary of all triggers using station.get_triggers()
trigger = station.get_first_trigger()

In [ ]:
# Let's plot the traces of the first four channels (=the phased array):
channel_ids = station.get_channel_ids()[:4]

fig, axs = plt.subplots(1, 2, figsize=(12,4))

for channel_id in channel_ids:
    channel = station.get_channel(channel_id)
    axs[0].plot(
        channel.get_times() / units.ns,
        channel.get_trace() / units.mV, # unit conversion is easy in NuRadioMC!
        # label=f'Ch. {channel_id}'
    )

    # plot the frequency spectra:
    axs[1].plot(
        channel.get_frequencies() / units.MHz,
        abs(channel.get_frequency_spectrum()) / (units.V / units.GHz),
        label=f'Ch. {channel_id}'
    )

# We'll also plot the trigger time:
trigger_time = trigger.get_trigger_time()
trigger_name = trigger.get_name()
axs[0].axvline(trigger_time, ls=':', color='grey', label=trigger_name)


axs[0].legend()
axs[1].legend()
axs[0].set_xlabel('t [ns]')
axs[1].set_xlabel('f [MHz]')
axs[0].set_ylabel('voltage [mV]')
axs[1].set_ylabel('amplitude [V/GHz]')

plt.show()

**Accessing parameters**
-------------------------------------

Parameters (neutrino energy, direction, Efield polarization...) are stored using the [`NuRadioReco.framework.parameters`](https://nu-radio.github.io/NuRadioMC/NuRadioReco/apidoc/NuRadioReco.framework.parameters.html#module-NuRadioReco.framework.parameters) class. They can be obtained by calling `.get_parameter(parameter_name)` on the relevant object, and similarly **set** by using `.set_parameter(parameter_name)`.

For a full list of parameters currently defined in `NuRadioMC`, consult the [`documentation`](https://nu-radio.github.io/NuRadioMC/NuRadioReco/apidoc/NuRadioReco.framework.parameters.html#module-NuRadioReco.framework.parameters).

In [ ]:
# we have to import the parameters to be able to use them!
from NuRadioReco.framework import parameters

The [`stationParameters`](https://nu-radio.github.io/NuRadioMC/NuRadioReco/apidoc/NuRadioReco.framework.parameters.html#NuRadioReco.framework.parameters.stationParameters) can also be accessed array-style from the `.nur` header:

In [ ]:
# this contains the maximum amplitude, over all channels, per event
station_id = 11 # the station_id of the station we are interested in
max_amplitudes = header[station_id][parameters.stationParameters.channels_max_amplitude]
max_amplitudes

Other parameters need to be called individually using the `.get_parameter` syntax.

In [ ]:
particle =  event.get_primary() # this returns the primary particle
# in the case of mu / tau, one can access all particles in the event decay chain
# by iterating over event.get_particles()

for par in parameters.particleParameters:
    if particle.has_parameter(par):
        print(par, particle.get_parameter(par))

In [ ]:
station = event.get_station()

for par in parameters.stationParameters:
    if station.has_parameter(par):
        print(par, station.get_parameter(par))


Let's have a look at what's stored in channel 0:

In [ ]:
channel = station.get_channel(0)

for par in parameters.channelParameters:
    if channel.has_parameter(par):
        print(par, channel.get_parameter(par))


As we are looking at a `.nur` file from simulation, which we haven't run any reconstruction on yet, many parameters in the [`Station`](https://nu-radio.github.io/NuRadioMC/NuRadioReco/apidoc/NuRadioReco.framework.station.html#module-NuRadioReco.framework.station) or [`Channel`](https://nu-radio.github.io/NuRadioMC/NuRadioReco/apidoc/NuRadioReco.framework.channel.html#module-NuRadioReco.framework.channel) (which are either used in or output during reconstruction), are undefined. We can have a look at some interesting quantities at MC-truth instead by looking at the [`SimStation`](https://nu-radio.github.io/NuRadioMC/NuRadioReco/apidoc/NuRadioReco.framework.sim_station.html#module-NuRadioReco.framework.sim_station). 

The syntax is similar, but not identical to the one for the [`Station`](https://nu-radio.github.io/NuRadioMC/NuRadioReco/apidoc/NuRadioReco.framework.station.html#module-NuRadioReco.framework.station). This is because, for example, multiple showers, and multiple ray tracing solutions mean that multiple voltage traces exist for each channel. We can access all of these by using the provided iterators:

In [ ]:
# First, we get the SimStation, which is stored inside the Station:
sim_station = station.get_sim_station()
# plot the MC voltage traces (no noise) for channel 0
channel_id = 0

for sim_channel in sim_station.get_channels_by_channel_id(channel_id):
    plt.plot(
        sim_channel.get_times() / units.ns,
        sim_channel.get_trace() / units.mV,
        label=(
            f'shower {sim_channel.get_shower_id()} / '
            f'RT solution {sim_channel.get_ray_tracing_solution_id()}'
        ))

plt.ylabel('Amplitude [mV]')
plt.xlabel('t [ns]')
plt.legend()

Now let's try to look at the electric field that caused these pulses:

In [ ]:
# get all electric fields for channel `channel_id`
efields = list(sim_station.get_electric_fields_for_channels(channel_ids=[channel_id]))

Let's first look at which parameters are stored in one of these electric fields:

In [ ]:
efield = efields[0] # let's look at the first electric field

print(f"Electric field for shower {efield.get_shower_id()} and ray tracing solution {efield.get_ray_tracing_solution_id()}.")
for par in parameters.electricFieldParameters:
    if efield.has_parameter(par):
        print(par, efield.get_parameter(par))


The `zenith` and `azimuth` above correspond to the **receiving** angle at the antenna - this is relevant for the antenna response. The [`ElectricField`](https://nu-radio.github.io/NuRadioMC/NuRadioReco/apidoc/NuRadioReco.framework.electric_field.html#module-NuRadioReco.framework.electric_field) object itself also contains traces - but this time they're electric field traces, polarized in spherical coordinates:

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(12,4), sharey=True, sharex=True)

polarizations = ['$E_R$', r'$E_\theta$', '$E_{\phi}$']
# The default coordinate system for electric fields.
# To convert to cartesian coordinates, or vxB x vxvxB,
# you can use the radiotools.coordinatesystems module

for i in range(3):
    axs[i].plot(efield.get_times(), efield.get_trace()[i] / (units.mV / units.m))
    axs[i].set_xlabel('t [ns]')
    axs[i].set_title(polarizations[i])

axs[0].set_ylabel('Electric Field Amplitude [mV/m]')
plt.tight_layout()
plt.show()

Finally, we will have a look at the `SimShower` that caused this electric field.

In [ ]:
shower_id = efield.get_shower_id()

sim_shower = event.get_sim_shower(shower_id)
print(f'Looking at shower with id {shower_id}')

# we can also get the first sim shower using event.get_first_sim_shower(),
# or iterate over all sim showers using event.get_sim_showers()

for par in parameters.showerParameters:
    if sim_shower.has_parameter(par):
        print(par, sim_shower.get_parameter(par))
